# Robotic fabrication fundamentals II

Path planning and assembly process simulation 

## Motion planning

* Collision checking (= path planning)
* Trajectory checking (synchronization, consider speed and acceleration of moving objects) 

<img src="images/path_planning_00.png" width="600" />

# Collision checking

* Intricate positions (spatial assembly)
* Multiple robots working closely

<table>
<tr>
<td><img src="http://www.dfab.arch.ethz.ch/data/ProjectImages/02_Web/M/248/190617_248_robotic_setup_3x2_AX_WM.jpg" style="height: 300px" /></td>
<td><img src="http://www.dfab.arch.ethz.ch/data/ProjectImages/02_Web/M/188/160922_188_BuildupPrototype_MP_01_WM.jpg" style="height: 300px" /></td>
</tr>
</table>



# Trajectory checking

* Synchronisation
* Continuous processes

<table>
<tr>
<td><img src="images/path_planning_04.jpg" style="height: 300px" /></td>
<td><img src="http://www.dfab.arch.ethz.ch/web/images/content/GKR_Infrastructure_7.jpg" style="height: 300px" /></td>
</tr>
</table>


## Path vs. Trajectory

### Path
A sequence of robot configurations in a particular order without regard to the timing between these configurations
(geometric path, obstacle avoidance, shortest path).

### Trajectory
Is concerned about the timing between these configurations (involving accelleration, speed, limits).

In [21]:
"""Example: Joint Trajectory Point and Joint Trajectory
"""

from compas.robots import Joint
from compas_fab.robots import Configuration
from compas_fab.robots import JointTrajectoryPoint

# create configuration
values = [1.571, 0, 0, 0.262, 0, 0]
types = [Joint.REVOLUTE] * 6
c = Configuration(values, types)

# create joint trajectory point
p = JointTrajectoryPoint(values, types)
print(p.accelerations)
print(p.velocities)
print(p.time_from_start)

# create joint trajectory
trj = JointTrajectory([p])
print(trj)


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Duration(0, 0)


## MoveIt!

* ROS’ default motion planning library
* Works with different planners: OMPL (Open Motion Planning Library)
* Collision checking through FCL (Flexible Collision Library)
* Kinematics plugins (default KDL: Kinematics and Dynamics Library)
* Trajectory processing routine

<img src="https://moveit.ros.org/assets/images/moveit2_logo_black.png" width="300">


In [ ]:
https://gramaziokohler.github.io/compas_fab/latest/backends/ros.html

docker compose up

## RViz in web browser


http://localhost:8080/vnc.html?resize=scale&autoconnect=true

<img src="images/moveit_ur5.jpg" width="700" />

<img src="images/moveit_rfl.jpg" width="700" />

<img src="images/moveit_ddad.jpg" width="700" />

In [ ]:
list services via roslibpy

In [ ]:
## Robot class in compas_fab

- model
- artist
- semantic
- client

In [23]:
import os
import compas
import compas_fab
from compas.robots import *
from compas_fab.robots import *
from threejs_viewer import RobotArtist

urdf_filename = compas_fab.get('universal_robot/ur_description/urdf/ur5.urdf')
srdf_filename = compas_fab.get('universal_robot/ur5_moveit_config/config/ur5.srdf')

model = RobotModel.from_urdf_file(urdf_filename)
loader = LocalPackageMeshLoader(compas_fab.get('universal_robot'), 'ur_description')
model.load_geometry(loader)

semantics = RobotSemantics.from_srdf_file(srdf_filename, model)
artist = RobotArtist(model)

robot = Robot(model, artist=artist, semantics=semantics)
print("done")
    

done


In [24]:
robot.info()

The robot's name is 'ur5'.
The planning groups are: ['manipulator', 'endeffector']
The main planning group is 'manipulator'.
The end-effector's name is 'ee_link'.
The base link's name is 'base_link'
The base_frame is: Frame(Point(0.000, 0.000, 0.000), Vector(1.000, 0.000, 0.000), Vector(-0.000, 1.000, 0.000))
The robot's joints are:
	* 'shoulder_pan_joint' is of type 'revolute' and has limits [6.283, -6.283]
	* 'shoulder_lift_joint' is of type 'revolute' and has limits [6.283, -6.283]
	* 'elbow_joint' is of type 'revolute' and has limits [6.283, -6.283]
	* 'wrist_1_joint' is of type 'revolute' and has limits [6.283, -6.283]
	* 'wrist_2_joint' is of type 'revolute' and has limits [6.283, -6.283]
	* 'wrist_3_joint' is of type 'revolute' and has limits [6.283, -6.283]
The robot's links are:
['base_link', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'ee_link', 'base', 'tool0', 'world']


## Inverse kinematic

In [22]:
from compas.geometry import Frame
from compas_fab.backends import RosClient
from compas_fab.robots import Configuration

frame = Frame([0.3, 0.1, 0.5], [1, 0, 0], [0, 1, 0])
start_configuration = Configuration.from_revolute_values([0] * 6)
group = "manipulator" # or robot.main_group_name

with RosClient() as client:
    robot.client = client
    config = robot.inverse_kinematics(frame, start_configuration, group)

print(config)




Twisted reactor is already running


AttributeError: module 'compas_fab.robots.robot' has no attribute 'inverse_kinematics'

In [5]:
from threejs_viewer import ThreeJsViewer

robot.update(config)

geo = robot.draw_visual()
viewer = ThreeJsViewer()
viewer.show(geo)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(intensity=0.5, position=(0.0, 0.0, 1.0), quaterni…

In [ ]:
# Cartesian path

* What is a cartesian path

In [ ]:
# Difference cartesian motion and free-space motion
joint space vs task space
robot following a straight line (robot runnning into itsel)


In [12]:
from compas.geometry import Frame
from compas_fab.backends import RosClient
from compas_fab.robots import Configuration

frames = []
frames.append(Frame([0.3, 0.1, 0.5], [1, 0, 0], [0, 1, 0]))
frames.append(Frame([0.4, 0.3, 0.4], [0, 1, 0], [0, 0, 1]))
start_configuration = Configuration.from_revolute_values([-0.042, 0.033, -2.174, 5.282, -1.528, 0.000])

with RosClient() as client:
    robot.client = client
    trajectory = robot.plan_cartesian_motion(frames,
                                             start_configuration,
                                             max_step=0.01,
                                             avoid_collisions=True)

print("Computed cartesian path with %d configurations, " % len(trajectory.points))
print("following %d%% of requested trajectory." % (trajectory.fraction * 100))
print("Executing this path at full speed would take approx. %.3f seconds." % trajectory.time_from_start)

Twisted reactor is already running


Computed cartesian path with 23 configurations, 
following 85% of requested trajectory.
Executing this path at full speed would take approx. 1.624 seconds.


# Free-space motion

The problem of motion planning can be stated as follows. 

Given:
* A start pose of the robot
* A desired goal pose
* A geometric description of the robot
* A geometric description of the world

Find a path that moves the robot gradually from start to goal while never touching any obstacle 

<img src="http://www.willowgarage.com/sites/default/files/blog/200909/path_planning_01.600h.png" />

In [16]:
import math
from compas.geometry import Frame
from compas_fab.robots import Configuration
from compas_fab.backends import RosClient

frame = Frame([0.4, 0.3, 0.4], [0, 1, 0], [0, 0, 1])
tolerance_position = 0.001
tolerance_axes = [math.radians(1)] * 3

start_configuration = Configuration.from_revolute_values([-0.042, 4.295, 0, -3.327, 4.755, 0.])
group = robot.main_group_name

# create goal constraints from frame
goal_constraints = robot.constraints_from_frame(frame,
                                                tolerance_position,
                                                tolerance_axes,
                                                group)

with RosClient() as client:
    robot.client = client
    
    trajectory = robot.plan_motion(goal_constraints,
                                   start_configuration,
                                   group,
                                   planner_id='RRT')

print("Computed kinematic path with %d configurations." % len(trajectory.points))
print("Executing this path at full speed would take approx. %.3f seconds." % trajectory.time_from_start)

Twisted reactor is already running


Computed kinematic path with 25 configurations.
Executing this path at full speed would take approx. 4.179 seconds.


In [ ]:
<iframe src="http://localhost:8080/vnc.html?resize=scale&autoconnect=true">
</iframe>